## Interval Analysis

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()


Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [2]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [3]:
train_model(model, 15)
test_model(model)

Epoch 1/15, Loss: 2.012
Epoch 2/15, Loss: 1.729
Epoch 3/15, Loss: 1.627
Epoch 4/15, Loss: 1.596
Epoch 5/15, Loss: 1.582
Epoch 6/15, Loss: 1.572
Epoch 7/15, Loss: 1.566
Epoch 8/15, Loss: 1.561
Epoch 9/15, Loss: 1.557
Epoch 10/15, Loss: 1.553
Epoch 11/15, Loss: 1.550
Epoch 12/15, Loss: 1.547
Epoch 13/15, Loss: 1.545
Epoch 14/15, Loss: 1.542
Epoch 15/15, Loss: 1.540
Accuracy on images: 93.43


### Write the interval analysis for the simple model

In [8]:
# Install bound_propagation if needed
# !pip install bound_propagation

import numpy as np
from bound_propagation import BoundModelFactory, HyperRectangle

# Define a simple sequential network for bound propagation
# Note: We create a new model structure compatible with bound_propagation
class SimpleNet(nn.Sequential):
    def __init__(self):
        super().__init__(
            nn.Linear(28*28, 200),
            nn.ReLU(),
            nn.Linear(200, 10)
        )

# Create network and copy weights from trained model
net = SimpleNet()
with torch.no_grad():
    net[0].weight.copy_(model[1].fc.weight)
    net[0].bias.copy_(model[1].fc.bias)
    net[2].weight.copy_(model[1].fc2.weight)
    net[2].bias.copy_(model[1].fc2.bias)

net = net.to(device)
net.eval()

# Convert to bound-propagation capable network
factory = BoundModelFactory()
net = factory.build(net)

def compute_bounds(net, x, epsilon):
    """
    Compute interval bounds for the network output.
    
    Args:
        net: Bound-propagation capable network
        x: Input tensor (batch of images)
        epsilon: L-infinity perturbation radius
    
    Returns:
        Interval bounds for network output
    """
    # Flatten and normalize input
    x = x.view(x.shape[0], -1)
    x = (x - 0.1307) / 0.3081
    
    # Create hyperrectangle input bounds
    epsilon_normalized = epsilon / 0.3081
    input_bounds = HyperRectangle.from_eps(x, epsilon_normalized)
    
    # Perform interval bound propagation
    bounds = net.ibp(input_bounds)
    
    return bounds

def is_verified_robust(bounds, label):
    """
    Check if prediction is verifiably robust for given label.
    
    Args:
        bounds: Interval bounds object
        label: True label index
    
    Returns:
        Boolean indicating verified robustness
    """
    lower = bounds.lower
    upper = bounds.upper
    
    # Check if lower bound of true class exceeds upper bounds of all other classes
    label_lower = lower[0, label]
    
    for i in range(lower.shape[1]):
        if i != label and upper[0, i] >= label_lower:
            return False
    
    return True

def verify_test_set(net, loader, epsilon):
    """
    Verify robustness on entire test set.
    
    Args:
        net: Bound-propagation capable network
        loader: Data loader
        epsilon: Perturbation radius
    
    Returns:
        Tuple of (num_robust, num_total, accuracy)
    """
    num_robust = 0
    num_total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        for i in range(images.shape[0]):
            x = images[i:i+1]
            y = labels[i].item()
            
            # Only verify correctly classified samples
            with torch.no_grad():
                pred = model(x).argmax(dim=1).item()
            
            if pred == y:
                bounds = compute_bounds(net, x, epsilon)
                if is_verified_robust(bounds, y):
                    num_robust += 1
                num_total += 1
    
    accuracy = 100.0 * num_robust / num_total if num_total > 0 else 0.0
    return num_robust, num_total, accuracy

# Run verification on whole test set


# 10 epsilon values evenly spaced between 0.01 and 0.1 (inclusive)
epsilon_values = np.linspace(0.01, 0.1, 10)

results = []
for idx, eps in enumerate(epsilon_values, 1):
    print(f"[{idx}/10] Testing ε = {eps:.4f}...", end=" ")
    num_robust, num_total, certified_acc = verify_test_set(net, test_loader, eps)
    results.append((eps, num_robust, num_total, certified_acc))
    print(f"Verified: {num_robust}/{num_total} ({certified_acc:.2f}%)")



for eps, num_robust, num_total, certified_acc in results:
    print(f"{eps:<15.4f} {num_robust:<20d} {num_total:<15d} {certified_acc:<20.2f}%")

print("=" * 80)

[1/10] Testing ε = 0.0100... Verified: 1220/9343 (13.06%)
[2/10] Testing ε = 0.0200... Verified: 3/9343 (0.03%)
[3/10] Testing ε = 0.0300... Verified: 0/9343 (0.00%)
[4/10] Testing ε = 0.0400... Verified: 0/9343 (0.00%)
[5/10] Testing ε = 0.0500... Verified: 0/9343 (0.00%)
[6/10] Testing ε = 0.0600... Verified: 0/9343 (0.00%)
[7/10] Testing ε = 0.0700... Verified: 0/9343 (0.00%)
[8/10] Testing ε = 0.0800... Verified: 0/9343 (0.00%)
[9/10] Testing ε = 0.0900... Verified: 0/9343 (0.00%)
[10/10] Testing ε = 0.1000... Verified: 0/9343 (0.00%)
0.0100          1220                 9343            13.06               %
0.0200          3                    9343            0.03                %
0.0300          0                    9343            0.00                %
0.0400          0                    9343            0.00                %
0.0500          0                    9343            0.00                %
0.0600          0                    9343            0.00                %
0.070